In [26]:
import numpy as np
from numpy.linalg import det, inv
from scipy.special import digamma

In [191]:
N = 12 # length of history
p = 20 # number of song features
K = 10 # number of discrete intervals 
x = np.zeros((N, p))
t = np.zeros((N, K))
r = np.zeros(N)
THRESHOLD = 10e-3

In [281]:
class Bayes_UCB_V:
    def __init__(self, x, t, r):
        self.x = x # features of songs
        self.t = t # last listened times of songs
        self.r = r # ratings
        self.N = x.shape[0]
        self.p = x.shape[1]
        self.K = t.shape[1]
        # TODO: How to initialize these variables?
        self.lambda_theta_N = np.identity(self.p)
        self.eta_theta_N = np.zeros(self.p)
        self.eta_beta_N = np.zeros(self.K)
        self.lambda_beta_N = np.identity(self.K)
        self.D_0 = np.identity(self.p)
        self.E_0 = np.identity(self.K)
        self.mu_theta_0 = np.zeros(self.p)
        self.mu_beta_0 = np.zeros(self.K)
        self.a_0 = 2
        self.b_0 = 2 * 10e-8
        self.a_N = self.a_0
        self.b_N = self.b_0
        self.prev_L = 0.0

        
    def optimize_parameters(self):
        while self.is_converged(self.lower_bound()) == False:
            self.update_theta()
            self.update_beta()
            self.update_tau()
        return self.lambda_theta_N, self.eta_theta_N, self.lambda_beta_N, self.eta_beta_N
    
    def lower_bound(self):       
        return (self.a_0 * np.log(self.b_0) 
            + (self.a_0 - 1) * (digamma(self.a_N) - np.log(self.b_N))
            - self.b_0 * self.a_N / self.b_N
            - (self.p / 2) * np.log(2 * np.pi)
            - (1 / 2) * np.log(det(self.D_0))
            + (self.p / 2) * (digamma(self.a_N) - np.log(self.b_N))
            - (self.a_N / (2 * self.b_N)) * (np.trace(self.D_0 * inv(self.lambda_theta_N)) 
                                               + np.dot(np.dot((self.mu_theta_0 - self.expected_theta()).T, inv(self.D_0)) , (self.mu_theta_0 - self.expected_theta()))
                                             )
            - (self.K / 2) * np.log(2 * np.pi)
            - (1 / 2) * np.log(det(self.E_0))
            + (self.K / 2) * (digamma(self.a_N) - np.log(self.b_N))
            - (self.a_N / (2 * self.b_N)) * (np.trace(self.E_0 * inv(self.lambda_beta_N)) 
                                               + np.dot(np.dot((self.mu_beta_0 - self.expected_beta()).T, inv(self.E_0)) , (self.mu_beta_0 - self.expected_beta()))
                                             )
            - (1 / 2) * np.log(2 * np.pi)
            + (1 / 2) * (digamma(self.a_N) - np.log(self.b_N))
            - (self.a_N / (2 * self.b_N)) * self.sum1() 
            + (self.a_N / self.b_N) * self.sum2()
            + (self.K / 2) * (1 + np.log(2 * np.pi))
            + (1 / 2) * (np.log(det(inv(self.lambda_beta_N))))
            + (self.p / 2) * (1 + np.log(2 * np.pi))
            + (1 / 2) * (np.log(det(inv(self.lambda_theta_N))))
            - (self.a_N - 1) * digamma(self.a_N)
            - np.log(self.b_N) + self.a_N        
               )        
                       
    def sum1(self):
        result = np.empty((x.shape[0]))
        for i in range(x.shape[0]):
            result[i] = np.dot(np.dot(x[i], self.expected_outer_theta()), x[i].T) * np.dot(np.dot(t[i], self.expected_outer_beta()), t[i].T)
        return (self.r ** 2).sum() + result.sum()
    
    def sum2(self):
        result = np.empty((x.shape[0]))
        for i in range(x.shape[0]):
            result[i] = np.dot(np.dot(self.r[i], self.x[i]), self.expected_theta()) * np.dot(self.t[i], self.expected_beta())
        return result.sum()
                       
    def is_converged(self, L):
        if abs(L - self.prev_L) < THRESHOLD:
            return True
        else:
            self.prev_L = L
            return False
    
    def update_theta(self):
        self.lambda_theta_N = self.expected_tau() * (inv(self.D_0) + np.dot(np.dot(np.dot(self.x.T, self.t), self.expected_outer_beta()), np.dot(self.t.T, self.x)))              
        self.eta_theta_N = self.expected_tau() * (np.dot(inv(self.D_0), self.mu_theta_0)  + np.dot(np.multiply(self.r[:, np.newaxis], self.x).T, np.dot(self.t, self.expected_beta()))                                                 )

    def update_beta(self):
        #self.lambda_beta_N = self.expected_tau() * (inv(self.E_0) + self.t * self.x.T * self.expected_outer_theta() * self.x * self.t.T )               
        self.lambda_beta_N = self.expected_tau() * (inv(self.E_0) + np.dot(np.dot(np.dot(self.t.T, self.x), self.expected_outer_theta()), np.dot(self.x.T, self.t)))              
        self.eta_beta_N = self.expected_tau() * (np.dot(inv(self.E_0), self.mu_beta_0)  + np.dot(np.multiply(self.r[:, np.newaxis], self.t).T, np.dot(self.x, self.expected_theta())))

        #self.eta_beta_N = self.expected_tau() * (inv(self.E_0) * self.mu_beta_0  + self.r * self.t * self.x.T * self.expected_theta())
        print('update beta')
        print(self.lambda_beta_N.shape)
        print(self.eta_beta_N.shape)
    def update_tau(self):
        self.a_N = (self.p + self.K + self.N) / 2 + self.a_0
        self.b_N = ((1 / 2) * 
                    (np.trace(np.dot(inv(self.D_0), self.expected_outer_theta())) +
                     np.dot((self.mu_theta_0.T - (2 * self.expected_theta().T)), np.dot(inv(self.D_0), self.mu_theta_0))
                    ) +
                    (1 / 2) * 
                    (np.trace(np.dot(inv(self.E_0), self.expected_outer_beta())) +
                     np.dot((self.mu_beta_0.T - (2 * self.expected_beta().T)), np.dot(inv(self.E_0), self.mu_beta_0))
                    ) 
                    + (1 / 2) * self.sum1() -
                    self.sum2() + self.b_0
                   )

    def expected_beta(self):
        return np.dot(inv(self.lambda_beta_N), self.eta_beta_N)
    
    def expected_theta(self):
        return np.dot(inv(self.lambda_theta_N), self.eta_theta_N)
    
    def expected_tau(self):
        return self.a_N / self.b_N
    
    def expected_outer_beta(self):
        return inv(self.lambda_beta_N) + self.expected_beta() * self.expected_beta().T
    
    def expected_outer_theta(self):
        return inv(self.lambda_theta_N) + self.expected_theta() * self.expected_theta().T


In [282]:
bayesUCB_V = Bayes_UCB_V(x, t, r)
bayesUCB_V.variational_inference()

trace
(20,)
(20,)
(20,)
0.0
sum
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
L
-149999740.27908006
update theta
(20, 20)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(12, 20)
(20,)
(20,)
update beta
(10, 10)
(10,)
sum
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
trace
(20,)
(20,)
(20,)
0.0
sum
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
L
-43.17961658682593
update theta
(20, 20)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(12, 20)
(20,)
(20,)
update beta
(10, 10)
(10,)
sum
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
trace
(20,)
(20,)
(20,)
0.0
sum
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
L
-43.13587699828133
update theta
(20, 20)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(12, 20)
(20,)
(20,)
update beta
(10, 10)
(10,)
sum
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
trace
(20,)
(20,)
(20,)
0.0
sum
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
L
-43.26381179743021
update theta
(20, 20)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(12, 20)
(20,)
(20,)
update